In [1]:
import os
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dash_table
from dash.dash_table.Format import Group
from dash import html, dcc
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from navbar_tabs_layout import app_layout
import time
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta

/Users/laetitiahoquetis/Downloads/Python/Dashboard/navbar_tabs_layout.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/Users/laetitiahoquetis/Downloads/Python/Dashboard/navbar_tabs_layout.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
# Read the data
df = pd.read_csv('EZItotalFull.csv')

/var/folders/jt/qpx02c196772_ytczgpv4llh0000gp/T/ipykernel_30671/3951316332.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('EZItotalFull.csv')


In [3]:
# #connect to database
# # Import required libraries
# import pymysql
# pymysql.install_as_MySQLdb()
# import MySQLdb

# # Define a function to fetch data from the database
# def fetch_data():
#     # Connect to the database
#     conn = pymysql.connect(
#         user = 'readonly',
#         password = 'MwLvD9DFL8mnrTKkI6fU', 
#         host = '127.0.0.1',
#         charset='utf8mb4',
#         port=int(3335))
#         #database=database,
#         #port=int(3306))

#     cursor = conn.cursor()
    
#     # Fetch data from SQL queries
#     query1 = """
#         SELECT i.sales_order_id, i.id, i.quantity, i.product_id, i.state, i.status, i.created_at, i.external_unit_price,
#         p.id, p.sku,  p.name, p.description, p.currency, p.brand_id , p.face_price , p.country , b.name

#         FROM ezscm_production.sales_order_items as i
#         JOIN ezscm_production.products as p
#         ON p.id = i.product_id
#         JOIN ezscm_production.brands as b
#         ON p.brand_id = b.id
#         WHERE i.created_at BETWEEN '2019-01-01' AND '2023-12-31'
#         """
#     query2 = """
#         SELECT o.id , o.date, o.invoice_number, o.state, o.customer_company_id, 
#         c.id , c.name, a.city, a.country, x.finance_number, x.sales_order_id

#         FROM ezscm_production.sales_orders as o
#         JOIN ezscm_production.companies as c
#         ON o.customer_company_id = c.id

#         LEFT OUTER JOIN ezscm_production.addresses as a
#         ON c.billing_address_id = a.id

#         LEFT OUTER JOIN ezscm_production.invoices as x
#         ON o.invoice_number = x.finance_number

#         WHERE o.date BETWEEN '2019-01-01' AND '2023-12-31'
#         """
#     df1 = pd.read_sql(query1,conn)
#     df2 = pd.read_sql(query2,conn)
    
#     # Merge dataframes
#     dfa= pd.merge(
#         df1,
#         df,
#         how="outer",
#         left_on=df1["id"],
#         right_on = df["sales_order_id"],
#         sort=True,
#         suffixes=("_o", "_i"),
#         copy=True,
#         indicator=True,
#         validate=None,
#     )
    
#     df = dfa[dfa._merge == 'both']
#     # Close the database connection
#     conn.close()
    
#     return df

In [4]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.UNITED], meta_tags=[{"name": "viewport", "content": "width=device-width"}])
app.title = 'Dashboard'

In [5]:
# @app.callback(
#     Output("df", "data"),
#     Input("fetch-data-button", "n_clicks")
# )
# def fetch_data(n_clicks):
#     # your code to fetch data and create a pandas DataFrame goes here
#     df = pd.read_csv('EZItotalFull.csv')
#     return df.to_dict("records")


In [6]:
# @app.callback(
#     Output("output-div", "children"),
#     Input("some-input", "value"),
#     State("df", "data")
# )
# def process_data(input_value, df_data):
#     # your code to process data using the df_data goes here
#     ...


In [7]:
today_date = datetime.now().strftime("%Y-%m-%d")
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [34]:
navbar = dbc.Navbar(
    dbc.Container(
        [
            dbc.Row(
                [
                    dbc.Col(html.Img(src="db.png", height="20px"), width="auto", align="center"),
                    dbc.Col(dbc.NavbarBrand("My Dashboard", className="ml-2"), width="auto", align="center"),
                    dbc.Col(html.Div(id="today-date", children=today_date), width="auto", align="center"),
                    dbc.Col(dbc.Button("Fetch Data", id="fetch-data-button", color="primary", className="mr-1"), width="auto", align="end"),
                ],
                className="my-row",
                align="center",
            ),
            dbc.NavbarToggler(id="navbar-toggler"),
        ]
    ),
    color="light",
    dark=False,
    sticky="top",
)


In [35]:
# Define page layout
page_layout = dbc.Container([
    
    # Add top_level section
    dbc.Row([
        dbc.Col(html.H6('Top level', className='card-title mb-4')),
        dbc.Col(dcc.DatePickerSingle(
            id='date-picker',
            min_date_allowed=df['date'].min(),
            max_date_allowed=df['date'].max(),
            initial_visible_month=df['date'].max(),
            date=df['date'].max()
        ), width=3),
    ], justify="between", align="center", className='mb-4'),

    dbc.Card([
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader("Sales YtD"),
                        dbc.CardBody([
                            dcc.Graph(
                                id='sales-chart-2'
                            )
                        ])
                    ], className="h-100",style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}),#style={"height": "400px", "overflow": "auto"}
                ], sm=12, lg=4, className="mb-4"),
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader("Sales Mtd"),
                        dbc.CardBody([
                            dcc.Graph(
                                id='sales-chart'
                            )
                        ])
                    ], className="h-100",style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}), #style={"height": "400px", "overflow": "auto"}
                ], sm=12, lg=4, className="mb-4"),
                dbc.Col([
                    dbc.Row([
                        dbc.Col([
                            dbc.Card([
                                dbc.CardHeader("Overnight Orders, including not Complete"),
                                dbc.CardBody([
                                    html.P(id='overnight-sales')
                                ])
                            ]),
                        ], md=12),
                        dbc.Col([
                            dbc.Card([
                                dbc.CardHeader("Customers to chase"),
                                dbc.CardBody([
                                    dash_table.DataTable(
                                        id='customers-table',
                                        columns=[{"name": i, "id": i} for i in ['name_o', 'orders_count', 'first_order', 'last_order', 'longevity']],
                                        data=[]
                                    )
                                ])
                            ], className="h-100"),
                        ], md=12)
                    ])
                ], sm=12, lg=4, className="mb-4"),  
            ])
        ])
    ], className="rounded-0 border-0"),

    # Add rest of page layout here
    html.Div("Click on below tabs for further analysis 👇"),

], fluid=True, style={'padding': '2rem'})


In [36]:
# #displaying gauge instead of bar chart

def new_sales_chart(date, df):
    current_year = pd.Timestamp(date).year
    current_month = pd.Timestamp(date).month
    current_day = pd.Timestamp(date).day
    prev_year = current_year - 1
    two_year = current_year - 2
    
    ytd_start = pd.to_datetime(f"{current_year}-01-01")
    last_ytd_start = pd.to_datetime(f"{prev_year}-01-01")
    last_ytd_end = pd.to_datetime(f"{prev_year}-{current_month}-{current_day}")
    two_ytd_start = pd.to_datetime(f"{two_year}-01-01")
    two_ytd_end = pd.to_datetime(f"{two_year}-{current_month}-{current_day}")

    sales_ytd = round(df[(df['date'] >= ytd_start)]['total'].sum(),2)
    sales_ytd_formatted = "{:,.2f}".format(sales_ytd)
    sales_ytd_last_year = df[(df['date'] >=last_ytd_start)& (df['date'] <= last_ytd_end)]['total'].sum()
    sales_ytd_two_year = df[(df['date'] >=two_ytd_start)& (df['date'] <= two_ytd_end)]['total'].sum()
    percent_vs_last_year_to_date = round(sales_ytd/sales_ytd_last_year*100,2)
    percent_vs_two_year_to_date = sales_ytd/sales_ytd_two_year 

    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=percent_vs_last_year_to_date,
        gauge={
            'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': 'darkblue'},
            'bar': {'color': 'deepskyblue'},
            'bgcolor': 'white',
            'borderwidth': 2,
            'bordercolor': 'gray',
            'steps': [{'range': [0, 50], 'color': 'white'},
                      {'range': [50, 100], 'color': 'white'},
                      {'range': [100, 150], 'color': 'white'},
                      {'range': [1500, 200], 'color': 'white'}]
        },
        domain={'x': [0, 1], 'y': [0, 1]},
        #title={'text': "% of YtD vs past YtD sales"}
    ))

    # Add the sales_ytd string as a markdown string to the layout of the fig variable
    fig.update_layout(
        annotations=[
            go.layout.Annotation(
                x=0.5,
                y=1.2,
                showarrow=False,
                text=f"YTD Sales: $ {sales_ytd_formatted}",
                xref="paper",
                yref="paper",
                align='center',
                font=dict(size=18)
            ),
            go.layout.Annotation(
                x=0.5,
                y=-0.2,
                showarrow=False,
                text=f"{percent_vs_last_year_to_date}% generated vs past YtD sales",
                xref="paper",
                yref="paper",
                align='center',
                font=dict(size=18)
            )
        ]
    )
    
    return fig
#new_sales_chart(today_date, df)

In [37]:
#displaying a gauge instead of bar chart 
@app.callback(
    Output('sales-chart-2', 'figure'),
    [Input('date-picker', 'date')])
def updated_new_sales_chart(date):
    # Call the new_sales_chart function to generate the figure
    fig = new_sales_chart(date, df)
    
    return fig

In [38]:
def create_sales_chart(df, date):
    month = pd.Timestamp(date).to_pydatetime()
    sales_mtd = df[(df['month'] == month.month) & (df['year'] == month.year)]['total'].sum()
    sales_mtd_formatted = "{:,.2f}".format(sales_mtd)
    sales_mtd_last_year = df[(df['month'] == month.month) & (df['year'] == month.year - 1)]['total'].sum()
    sales_mtd_two_year = df[(df['month'] == month.month) & (df['year'] == month.year - 2)]['total'].sum()
    percent_vs_last_year_month = round(sales_mtd/sales_mtd_last_year*100,2)
    percent_vs_two_year_month = round(sales_mtd/sales_mtd_two_year*100,2)

    labels = ['vs 2Y ago', 'vs last year','MtD']
    category_names = [sales_mtd_two_year, sales_mtd_last_year, sales_mtd_formatted]

    trace = go.Bar(
        x=category_names,
        y=labels,
        orientation='h',
        text=category_names,
        textposition='auto',
        marker=dict(
            color='deepskyblue'
        )
    )

    data = [trace]
    layout = go.Layout(
        #xaxis=dict(title='Total Sales'),
        #yaxis=dict(title='Category'),
        margin=dict(l=1)
    )

    fig = go.Figure(data=data, layout=layout)
    return fig
create_sales_chart(df, today_date)

In [39]:
@app.callback(
    Output('sales-chart', 'figure'),
    [Input('date-picker', 'date')]
)
def update_sales_chart(date):
    month = pd.Timestamp(date).to_pydatetime()
    sales_chart = create_sales_chart(df, month)
    return sales_chart


In [40]:
today = pd.Timestamp.now().floor('D')
def prev_weekday(adate):
    adate -= timedelta(days=1)
    while adate.weekday() > 4: # Mon-Fri are 0-4
        adate -= timedelta(days=1)
    return adate
overnight_sales = df[(df['date'] >= prev_weekday(today))]['total'].sum()


In [41]:
@app.callback(
    Output('overnight-sales', 'children'),
    [Input('date-picker', 'date')]
)
def update_overnight_sales(date):
    # Calculate overnight sales based on selected date
    overnight_sales = round(df[(df['date'] >= pd.Timestamp(date).floor('D'))]['total'].sum(),2)
    overnight_sales = "{:,.2f}".format(overnight_sales)
    return f"Overnight sales = {overnight_sales}"
#update_overnight_sales(today_date)

In [42]:
# def get_customers_to_chase(df):
#     # Filter for customers who last ordered this year
#     current_year = datetime.now().year
#     # How long have we got each cust
#     customer_length = df.groupby('name_o').agg({'sales_order_id_o': 'nunique', 'date': ['min', 'max']})
#     customer_length['longevity'] = customer_length['date']['max'] - customer_length['date']['min']
#     customer_length = customer_length.reset_index()
#     customer_length.columns = ['name_o', 'orders_count', 'first_order', 'last_order', 'longevity']

#     filtered_df = customer_length.loc[(customer_length['last_order'].dt.year == current_year)]
#     sorted_df = filtered_df.sort_values('last_order').head(10)

#     # Format the longevity column as a string
#     sorted_df['longevity'] = sorted_df['longevity'].apply(lambda x: str(datetime.min + x))
    
#     return sorted_df.to_dict('records')
    
# #     fig = go.Figure(data=[go.Table(
# #         header=dict(values=list(sorted_df.columns),
# #                     fill_color='lightgray',
# #                     align='left'),
# #         cells=dict(values=[sorted_df.name_o, sorted_df.orders_count, sorted_df.first_order, sorted_df.last_order, sorted_df.longevity],
# #                    fill_color='white',
# #                    align='left'))
# #     ])

# #     return fig

# #     table = dash_table.DataTable(
# #         columns=[{"name": i, "id": i} for i in sorted_df.columns],
# #         data=sorted_df.to_dict('records')
# #     )

# #     return table


In [43]:
def get_customers_to_chase(df):
    # Filter for customers who last ordered this year
    current_year = datetime.now().year
    # How long have we got each cust
    customer_length = df.groupby('name_o').agg({'sales_order_id_o': 'nunique', 'date': ['min', 'max']})
    customer_length['longevity'] = customer_length['date']['max'] - customer_length['date']['min']
    customer_length = customer_length.reset_index()
    customer_length.columns = ['name_o', 'orders_count', 'first_order', 'last_order', 'longevity']

    filtered_df = customer_length.loc[(customer_length['last_order'].dt.year == current_year)]
    sorted_df = filtered_df.sort_values('last_order').head(10)

    # Format the longevity column as a string
    sorted_df['longevity'] = sorted_df['longevity'].apply(lambda x: str(datetime.min + x))
    
    return sorted_df


In [44]:
get_customers_to_chase(df)

,name_o,orders_count,first_order,last_order,longevity
87,JS Music Corporation,27,2019-01-02,2023-01-04,0005-01-03 00:00:00
31,Comercializadora SYP Limitada (EUR),6,2020-10-07,2023-01-09,0003-04-05 00:00:00
5,ABAAD ALJEEL DBA LIKE4CARD (EUR),33,2021-01-04,2023-01-10,0003-01-07 00:00:00
89,KorsIT,216,2019-01-07,2023-01-30,0005-01-24 00:00:00
90,KorsIT (CAD),20,2021-02-23,2023-01-30,0002-12-08 00:00:00
25,COLOMBIANUP SAS,20,2021-08-24,2023-02-01,0002-06-11 00:00:00
34,DEIMAN UNITED LTD,68,2019-01-07,2023-02-13,0005-02-07 00:00:00
111,Napan LLC,44,2021-08-27,2023-02-13,0002-06-20 00:00:00
32,ContentCard AG,201,2019-07-03,2023-02-15,0004-08-16 00:00:00
102,MOL AccessPortal Sdn Bhd,76,2019-01-17,2023-02-22,0005-02-06 00:00:00


In [45]:
@app.callback(
    Output('customers-table', 'data'),
    Input('df', '_'),
    State('df', 'data')
)
def update_customers_table(_, df):
    sorted_df = get_customers_to_chase(df)
    return sorted_df.to_dict('records')

In [46]:
update_customers_table(_,df)

[{'name_o': 'JS Music Corporation',
  'orders_count': 27,
  'first_order': Timestamp('2019-01-02 00:00:00'),
  'last_order': Timestamp('2023-01-04 00:00:00'),
  'longevity': '0005-01-03 00:00:00'},
 {'name_o': 'Comercializadora SYP Limitada (EUR)',
  'orders_count': 6,
  'first_order': Timestamp('2020-10-07 00:00:00'),
  'last_order': Timestamp('2023-01-09 00:00:00'),
  'longevity': '0003-04-05 00:00:00'},
 {'name_o': 'ABAAD ALJEEL DBA LIKE4CARD (EUR)',
  'orders_count': 33,
  'first_order': Timestamp('2021-01-04 00:00:00'),
  'last_order': Timestamp('2023-01-10 00:00:00'),
  'longevity': '0003-01-07 00:00:00'},
 {'name_o': 'KorsIT',
  'orders_count': 216,
  'first_order': Timestamp('2019-01-07 00:00:00'),
  'last_order': Timestamp('2023-01-30 00:00:00'),
  'longevity': '0005-01-24 00:00:00'},
 {'name_o': 'KorsIT (CAD)',
  'orders_count': 20,
  'first_order': Timestamp('2021-02-23 00:00:00'),
  'last_order': Timestamp('2023-01-30 00:00:00'),
  'longevity': '0002-12-08 00:00:00'},
 {'na

In [47]:
# @app.callback(
#     Output('customers-table', 'children'),
#     Input('update-button', '_'),
# )
# def update_customers_table(_):
#     # Call the function to create the DataTable
#     customers_table = get_customers_to_chase(df)
    
#     return customers_table


In [48]:
# update_customers_table(_)

In [49]:
# # Filter for customers who last ordered this year
# current_year = datetime.now().year
# #How long have we got each cust 
# customer_length = df.groupby('name_o').agg({'sales_order_id_o': 'nunique','date': ['min', 'max']})
# customer_length['longevity'] = customer_length['date']['max'] - customer_length['date']['min']
# customer_length = customer_length.reset_index()
# customer_length.columns = ['name_o', 'orders_count', 'first_order', 'last_order','longevity']
# filtered_df = customer_length.loc[(customer_length['last_order'].dt.year == current_year)]
# filtered_df.sort_values('last_order').head(10)

In [50]:
# # Define Tabs content
# sales_tab_content = dbc.Card(
#     dbc.CardBody("tab to fill per year/period")
# )


In [51]:
# Sales dashboard
sales_tab_content = dbc.Container([
     dbc.Row([
        dbc.Col(html.H6('...', className='card-title mb-4')),
        html.Label('Select year:'), 
        dcc.Dropdown(
                            id='year-dropdown',
                            options=[{'label': str(year), 'value': year} for year in df['year'].unique()],
                            value=df['year'].min()
                        ),
        ], justify="between", align="center", className='mb-4'),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Sales by Top 3 Customers")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph-1')
                    ], 
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Sales vs Order vs Total SKU")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph-2')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Average Order Value")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph-3')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Sales by Region")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph-4')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
    ]),
], fluid=True, style={'padding': '2rem'})


In [52]:
# Calculate total sales by customer over the entire period
total_sales_by_customer = df.groupby('name_o')['total'].sum()

# Select top 20 customers based on their total sales
top_customers = total_sales_by_customer.nlargest(20).index.tolist()

# Create a dictionary of colors for the top 20 customers
customer_colors = {customer: px.colors.qualitative.Pastel1[i % len(px.colors.qualitative.Pastel1)]
                   for i, customer in enumerate(top_customers)}

# Add a color for 'others'
customer_colors['others'] = 'gray'

# Assign colors to each customer using the color mapping dictionary
def assign_color(customer):
    if customer in customer_colors:
        return customer_colors[customer]
    else:
        return 'gray'

@app.callback(
    Output('sales-graph-1', 'figure'),
    [Input('year-dropdown', 'value')])
def update_sales_graph_1(selected_year):
    filtered_df = df[df['year'] == selected_year]

    # Calculate total sales by month and customer
    sales_by_month_customer = filtered_df.groupby(['month', 'name_o'])['total'].sum().reset_index()

    # Find top customers for each month
    top_customers_by_month = sales_by_month_customer.groupby('month').apply(lambda x: x.nlargest(3, 'total')).reset_index(drop=True)

    # Group the remaining customers as "others"
    remaining_customers_by_month = sales_by_month_customer[~sales_by_month_customer['name_o'].isin(top_customers)]
    remaining_customers_by_month = remaining_customers_by_month.groupby('month').sum().reset_index()
    remaining_customers_by_month['name_o'] = 'others'

    # Concatenate the top customers and the "others" dataframes
    sales_by_month_customer = pd.concat([top_customers_by_month, remaining_customers_by_month])

    # Assign colors to each customer using the color mapping dictionary
    sales_by_month_customer['color'] = sales_by_month_customer['name_o'].apply(assign_color)

    # Create stacked bar chart of sales by customer
    fig = px.bar(sales_by_month_customer, x='month', y='total', color='name_o', color_discrete_map=customer_colors)
    fig.update_layout(
        title=f'Top 3 customers and others by month ({selected_year})',
        xaxis_title='Month',
        yaxis_title='Total Sales ($)',
        barmode='stack'
    )
    return fig
#update_sales_graph_1(2021)

In [53]:
@app.callback(
    Output('sales-graph-2', 'figure'),
    [Input('year-dropdown', 'value')])
def update_sales_graph_2(selected_year):
    filtered_df = df[df['year'] == selected_year]
    
    # Total Sales trend by month
    sales_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M')).sum()['total']
    sales_trace = go.Scatter(x=sales_by_month.index, y=sales_by_month.values, mode='lines', name='Total Sales')

    # Total Unique Orders trend by month
    unique_orders_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M'))['id_o'].nunique()
    unique_orders_trace = go.Scatter(x=unique_orders_by_month.index, y=unique_orders_by_month.values, mode='lines', name='Total Unique Orders')

    # Total SKU trend by month
    total_sku_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M'))['quantity'].sum()
    total_sku_trace = go.Scatter(x=total_sku_by_month.index, y=total_sku_by_month.values, mode='lines', name='Total SKUs')
    
    fig = go.Figure()
    
    fig.add_trace(sales_trace)
    fig.add_trace(unique_orders_trace)
    fig.add_trace(total_sku_trace)

    fig.update_layout(
        font=dict(size=10),
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="white",
        plot_bgcolor="white",
        xaxis_title='Month',
        yaxis_title='Total',
        title="Sales, Unique Orders, and SKUs Trend"
    )

    return fig
#update_sales_graph_2(2021)

In [54]:
clients_tab_content = dbc.Card(
    dbc.CardBody("tab to fill per client")
)

In [55]:
prospects_tab_content = dbc.Card(
    dbc.CardBody("habits and nudge")
)

In [56]:
# Define tabs
tabs = dbc.Tabs(
    [
        dbc.Tab(sales_tab_content, label="Historic Sales"),
        dbc.Tab(clients_tab_content, label="Customer Analysis"),
        dbc.Tab(prospects_tab_content, label="Prospects Market Research"),
    ],
    id="tabs",
    active_tab="sales_tab_content",
)

In [57]:
# Define app layout
app.layout = html.Div([
    navbar,
    dcc.Store(id="df"),
    page_layout,
    dbc.Container([
        dbc.Row([
            dbc.Col([
                tabs,
            ]),
        ], className='mt-4')
    ])
], style={'padding': '0rem 0rem 2rem 0rem'})

In [58]:
# # Update the page content based on the url
# @app.callback(Output("page-content", "children"), [Input("url", "pathname")])
# def render_page_content(pathname):
#     if pathname == "/":
#         return landing_page
#     elif pathname == "/apps/sales":
#         return sales_dashboard
#     elif pathname == "/apps/clients":
#         return clients_dashboard
#     elif pathname == "/apps/prospects":
#         return prospects_dashboard
#     else:
#         return html.P("Page not found")


In [59]:
# Set the app layout
#app.layout = app_layout
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
